In [1]:
import pandas as pd
from utils.esutils import query_enterprises_scroll
import re
import utils.processes as processes

In [2]:
enterprises = query_enterprises_scroll(province="江苏省")

已加载 100000 条记录...
已加载 200000 条记录...
已加载 300000 条记录...
已加载 400000 条记录...
已加载 500000 条记录...
已加载 600000 条记录...
已加载 700000 条记录...
已加载 800000 条记录...
已加载 900000 条记录...
已加载 1000000 条记录...
已加载 1100000 条记录...
已加载 1200000 条记录...
已加载 1300000 条记录...
已加载 1400000 条记录...
已加载 1500000 条记录...
已加载 1600000 条记录...
已加载 1700000 条记录...
已加载 1800000 条记录...
已加载 1900000 条记录...
已加载 2000000 条记录...
已加载 2100000 条记录...
已加载 2200000 条记录...
已加载 2300000 条记录...
已加载 2400000 条记录...
已加载 2500000 条记录...
已加载 2600000 条记录...
已加载 2700000 条记录...
已加载 2800000 条记录...
已加载 2900000 条记录...
已加载 3000000 条记录...
已加载 3100000 条记录...
已加载 3200000 条记录...
已加载 3300000 条记录...
已加载 3400000 条记录...
已加载 3500000 条记录...
已加载 3600000 条记录...
已加载 3700000 条记录...
已加载 3800000 条记录...
已加载 3900000 条记录...
已加载 4000000 条记录...
已加载 4100000 条记录...
已加载 4200000 条记录...
已加载 4300000 条记录...
已加载 4400000 条记录...
已加载 4500000 条记录...
已加载 4600000 条记录...
已加载 4700000 条记录...
已加载 4800000 条记录...
已加载 4900000 条记录...
已加载 5000000 条记录...
已加载 5100000 条记录...
已加载 5200000 条记录...
已加载 5300000 条记录...
已加

In [3]:
def deal(row):
        address = row["registered_address"]
        province = row["province"]
        city = row["city"]
        district = row["district"]

        row["extra_address"] = None
        row["matched_province"] = None
        row["matched_city"] = None
        row["matched_district"] = None

        if pd.isna(address) or address == "-":
            return row

        extra_address = address
        matched_province = None
        matched_city = None
        matched_district = None
        matched_town_or_street = None

        if not pd.isna(province) and province != "-" and province in extra_address:
            matched_province = province
            extra_address = extra_address.replace(province, "", 1)

        if not pd.isna(city) and city != "-" and city in extra_address:
            matched_city = city
            extra_address = extra_address.replace(city, "", 1)

        if not pd.isna(district) and district != "-" and district in extra_address:
            matched_district = district
            extra_address = extra_address.replace(district, "", 1)

         # 提取镇/街道（以“镇”或“街道”结尾的词）
        town_or_street_pattern = r"[^\s]{2,10}(?:镇|街道)"
        match = re.search(town_or_street_pattern, extra_address)
        if match:
            matched_town_or_street = match.group(0)
            extra_address = extra_address.replace(matched_town_or_street, "", 1)

        extra_address = extra_address.strip()
        if not extra_address:
            extra_address = None

        row["extra_address"] = extra_address
        row["matched_province"] = matched_province
        row["matched_city"] = matched_city
        row["matched_district"] = matched_district
        row["matched_town"] = matched_town_or_street
        
        # 进行预测，只保存预测错误的数据，减少存储压力
        

        return row

In [4]:
ok_df = processes.process_dataframe_multiprocess(df, deal, num_processes=4, chunk_size=100000)

处理进度 2025-04-05 10:35:40:0
处理进度 2025-04-05 10:35:40:100000
处理进度 2025-04-05 10:35:40:200000
处理进度 2025-04-05 10:35:40:300000
处理进度 2025-04-05 10:36:25:400000
处理进度 2025-04-05 10:36:25:500000
处理进度 2025-04-05 10:36:26:600000
处理进度 2025-04-05 10:36:27:700000
处理进度 2025-04-05 10:37:10:800000
处理进度 2025-04-05 10:37:11:900000
处理进度 2025-04-05 10:37:11:1000000
处理进度 2025-04-05 10:37:13:1100000
处理进度 2025-04-05 10:38:01:1200000
处理进度 2025-04-05 10:38:02:1300000
处理进度 2025-04-05 10:38:04:1400000
处理进度 2025-04-05 10:38:06:1500000
处理进度 2025-04-05 10:38:50:1600000
处理进度 2025-04-05 10:38:51:1700000
处理进度 2025-04-05 10:38:52:1800000
处理进度 2025-04-05 10:38:54:1900000
处理进度 2025-04-05 10:39:34:2000000
处理进度 2025-04-05 10:39:36:2100000
处理进度 2025-04-05 10:39:37:2200000
处理进度 2025-04-05 10:39:39:2300000
处理进度 2025-04-05 10:40:19:2400000
处理进度 2025-04-05 10:40:20:2500000
处理进度 2025-04-05 10:40:21:2600000
处理进度 2025-04-05 10:40:24:2700000
处理进度 2025-04-05 10:41:04:2800000
处理进度 2025-04-05 10:41:05:2900000
处理进度 2025-04-05 10:41:05:

In [14]:
def csv_to_bio(df, output_file):
    selected_columns = ["province", "city", "district", "matched_town", "extra_address"]
    
    with open(output_file, "w", encoding="utf-8") as f:
        for index, row in df.iterrows():
            # 处理每个字段
            for col in selected_columns:
                if pd.notna(row[col]) and (col != "matched_town" or row[col]):  # 跳过空的 matched_town
                    chars = list(row[col])  # 将字符串拆分为字符列表
                    if col == "province":
                        tag_prefix = "PROV"
                    elif col == "city":
                        tag_prefix = "CITY"
                    elif col == "district":
                        tag_prefix = "DISTRICT"
                    elif col == "matched_town":
                        tag_prefix = "TOWN"
                    elif col == "extra_address":
                        tag_prefix = "EXTRA"
                    
                    # 分配BIO标签
                    for i, char in enumerate(chars):
                        if i == 0:
                            f.write(f"{char} B-{tag_prefix}\n")
                        elif i == len(chars) - 1:
                            f.write(f"{char} E-{tag_prefix}\n")
                        else:
                            f.write(f"{char} I-{tag_prefix}\n")
            # 每条地址后加空行
            if index < len(df) - 1:
                f.write("\n")

# 生成train.txt
csv_to_bio(ok_df[ok_df['city'] == '无锡市'], "train无锡.txt")

In [15]:
len(ok_df[ok_df['city'] == '无锡市'])

1815829